In [1]:
# LH
###
noiselevel=0.01
###
from pyDOE import *
from freeCAD.my_API import get_displacement_magnitude

# EMU
from sklearn.metrics import r2_score

from freeCAD.my_API import get_displacement_magnitude
import numpy as np

import GPy
from emukit.model_wrappers.gpy_model_wrappers import GPyModelWrapper

from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop

from emukit.core import ParameterSpace, ContinuousParameter
from emukit.experimental_design.acquisitions import ModelVariance
from emukit.core.optimization import GradientAcquisitionOptimizer

import joblib
import sklearn
import pandas as pd
import normal

def get_displacement_magnitude_array(factors):
    factor = normal.un_nor_x(factors[0])
#     print(factor)
    result = get_displacement_magnitude(factor[2], factor[3], factor[4], factor[5], factor[6], factor[7],
                                        f"{factor[0]} MPa", str(factor[1]))
    valuerange=347.53-0.745 # testdataset 
    n=1  # training data 

    noise_dis= valuerange*np.random.random(size=n)*noiselevel # 

    direction=np.random.randint(0,2,n) # 单向?

    noise=[]
    for i in range(direction.shape[0]):
        if direction[i]==0:
            noise.append(0-noise_dis[i])
        else:
            noise.append(noise_dis[i])   
    samp_displacement_withnoise=result+noise[0]
    y = samp_displacement_withnoise
#     y = normal.nor_y(samp_displacement_withnoise)
#     print(y)
    return np.array([[y]])

# data
Ym_min = 60000
Ym_max = 500000

Pr_min = 0.1
Pr_max = 0.45

L_min = 8000
L_max = 10000

W_min = 1000
W_max = 2000

F1_min = 1000000
F1_max = 10000000

F2_min = 1000000
F2_max = 5000000

F3_min = 1000000
F3_max = 10000000

F4_min = 1000000
F4_max = 5000000


In [2]:
FREECADPATH = 'C:/Program Files/FreeCAD 0.20/bin/' #C:\Program Files\FreeCAD 0.20\bin\
import sys
sys.path.append(FREECADPATH)
import FreeCADGui
import FreeCAD  
import ObjectsFem 
import Fem 
from femtools import ccxtools 

In [3]:
def LH_generation(data_number, save):
    samples = lhs(8, data_number,criterion='centermaximin')  
    with open('C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_{}.csv'.format(save), "w") as f:
        f.write("Ym, Pr, L, W, F1, F2, F3, F4, Displacement\n")
        print('please wait for the LH generation')
        count = 0
        for sample in samples:
            samp_ym = Ym_min + (Ym_max - Ym_min) * sample[0]
            samp_pr = Pr_min + (Pr_max - Pr_min) * sample[1]
            samp_l = L_min + (L_max - L_min) * sample[2]
            samp_w = W_min + (W_max - W_min) * sample[3]
            samp_f1 = F1_min + (F1_max - F1_min) * sample[4]
            samp_f2 = F2_min + (F2_max - F2_min) * sample[5]
            samp_f3 = F3_min + (F3_max - F3_min) * sample[6]
            samp_f4 = F4_min + (F4_max - F4_min) * sample[7]

            samp_displacement = get_displacement_magnitude(samp_l, samp_w, samp_f1, samp_f2, samp_f3, samp_f4, f"{samp_ym} MPa", str(samp_pr))
            f.write(str(samp_ym) + "," + str(samp_pr) + "," + str(samp_l) + "," + str(samp_w) + "," + str(samp_f1) + "," + str(samp_f2) + "," + str(samp_f3) + "," + str(samp_f4) + "," + str(samp_displacement) + "\n")
    
    print('LH database has been saved successfully')

In [4]:
def EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = 45):
    
    #train_data
    X_init = []
    Y_init = []

    data = np.loadtxt(open(train_data, "r"), delimiter=",", skiprows=1)
    for row in data:
        Ym = row[0]
        Pr = row[1]
        L = row[2]
        W = row[3]
        F1 = row[4]
        F2 = row[5]
        F3 = row[6]
        F4 = row[7]
        X_init.append(normal.nor_x([Ym, Pr, L, W, F1, F2, F3, F4]))
        Y_init.append([row[8]])

    X_train = np.array(X_init)
    Y_train = np.array(Y_init)   
    
    #test_data
    X_init = []
    Y_init = []

    data = np.loadtxt(open(test_data, "r"), delimiter=",", skiprows=1)
    for row in data:
        Ym = row[0]
        Pr = row[1]
        L = row[2]
        W = row[3]
        F1 = row[4]
        F2 = row[5]
        F3 = row[6]
        F4 = row[7]
        X_init.append(normal.nor_x([Ym, Pr, L, W, F1, F2, F3, F4]))
        Y_init.append([row[8]])

    X_test = np.array(X_init)
    Y_test = np.array(Y_init)
    
    # model generation
    parameter_space = ParameterSpace([ContinuousParameter('Ym', 0, 1),
                                  ContinuousParameter('Pr', 0, 1),
                                  ContinuousParameter('L', 0, 1),
                                  ContinuousParameter('W', 0, 1),
                                  ContinuousParameter('F1', 0, 1),
                                  ContinuousParameter('F2', 0, 1),
                                  ContinuousParameter('F3', 0, 1),
                                  ContinuousParameter('F4', 0, 1),
                                  ])


    ker = GPy.kern.Bias(input_dim=8) + GPy.kern.Bias(1.0) * GPy.kern.RBF(input_dim=8, variance=1., lengthscale=1.,ARD=True) + GPy.kern.White(1) # R2=0.87
    # 生成GP高斯模型
    gpy_model = GPy.models.GPRegression(X_train, Y_train, ker)

    gpy_model.optimize(max_f_eval=10000)

    y_pre = gpy_model.predict_quantiles(X_test)[0]
    print('Gaussian process regression, R2=%.2f' % r2_score(Y_test, y_pre))

    # 生成emukit模型
    emukit_model = GPyModelWrapper(gpy_model)
    
    # 迭代
    model_variance = ModelVariance(model = emukit_model)

    ed = ExperimentalDesignLoop(space=parameter_space, model=emukit_model, acquisition = model_variance)
    max_iterations = iterations
    print('please wait fot the iteration')
    ed.run_loop(get_displacement_magnitude_array, max_iterations)

    y_pre_emu = ed.model.predict(X_test)[0]
    print('Gaussian process regression after iterations, R2=%.2f' % r2_score(Y_test, y_pre_emu))
    
    # data_base
    X_emu = ed.loop_state.X
    Y_emu = ed.loop_state.Y

    with open('C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\Emukit_noisein\\EMU_{}.csv'.format(data_name), "w") as f:
        f.write("Ym, Pr, L, W, F1, F2, F3, F4, Displacement\n")
        i = 0
        for line in X_emu:
            line = normal.un_nor_x(line)
            dis = Y_emu[i][0]
            f.write(
                str(line[0]) + "," + str(line[1]) + "," + str(line[2]) + "," + str(line[3]) + "," + str(line[4]) + "," + str(
                    line[5]) + "," + str(line[6]) + "," + str(line[7]) + "," + str(dis) + "\n")
            i = i + 1
    
    print('GPY database has been saved successfully')    
    
    with open('C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\Emukit_noisein\\GPY_model_with_{}.dat'.format(save_name),'wb') as f:
        joblib.dump(ed.model, f)   
    print('GPY model has been saved successfully') 

In [ ]:
# 保存GPy models

def EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = 45):
    
    #train_data
    X_init = []
    Y_init = []

    data = np.loadtxt(open(train_data, "r"), delimiter=",", skiprows=1)
    for row in data:
        Ym = row[0]
        Pr = row[1]
        L = row[2]
        W = row[3]
        F1 = row[4]
        F2 = row[5]
        F3 = row[6]
        F4 = row[7]
        X_init.append(normal.nor_x([Ym, Pr, L, W, F1, F2, F3, F4]))
        Y_init.append([row[8]])

    X_train = np.array(X_init)
    Y_train = np.array(Y_init)   
    
    #test_data
    X_init = []
    Y_init = []

    data = np.loadtxt(open(test_data, "r"), delimiter=",", skiprows=1)
    for row in data:
        Ym = row[0]
        Pr = row[1]
        L = row[2]
        W = row[3]
        F1 = row[4]
        F2 = row[5]
        F3 = row[6]
        F4 = row[7]
        X_init.append(normal.nor_x([Ym, Pr, L, W, F1, F2, F3, F4]))
        Y_init.append([row[8]])

    X_test = np.array(X_init)
    Y_test = np.array(Y_init)
    
    # model generation
    parameter_space = ParameterSpace([ContinuousParameter('Ym', 0, 1),
                                  ContinuousParameter('Pr', 0, 1),
                                  ContinuousParameter('L', 0, 1),
                                  ContinuousParameter('W', 0, 1),
                                  ContinuousParameter('F1', 0, 1),
                                  ContinuousParameter('F2', 0, 1),
                                  ContinuousParameter('F3', 0, 1),
                                  ContinuousParameter('F4', 0, 1),
                                  ])


    ker = GPy.kern.Bias(input_dim=8) + GPy.kern.Bias(1.0) * GPy.kern.RBF(input_dim=8, variance=1., lengthscale=1.,ARD=True) + GPy.kern.White(1) # R2=0.87
    # 生成GP高斯模型
    gpy_model = GPy.models.GPRegression(X_train, Y_train, ker)

    gpy_model.optimize(max_f_eval=10000)

    y_pre = gpy_model.predict_quantiles(X_test)[0]
    print('Gaussian process regression, R2=%.2f' % r2_score(Y_test, y_pre))

    # 生成emukit模型
    emukit_model = GPyModelWrapper(gpy_model)
    
    # 迭代
    model_variance = ModelVariance(model = emukit_model)

    ed = ExperimentalDesignLoop(space=parameter_space, model=emukit_model, acquisition = model_variance)
    max_iterations = iterations
    print('please wait fot the iteration')
    ed.run_loop(get_displacement_magnitude_array, max_iterations)

    y_pre_emu = ed.model.predict(X_test)[0]
    print('Gaussian process regression after iterations, R2=%.2f' % r2_score(Y_test, y_pre_emu))
    
    # data_base
    X_emu = ed.loop_state.X
    Y_emu = ed.loop_state.Y

    with open('C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\Emukit_noisein\\EMU_{}.csv'.format(data_name), "w") as f:
        f.write("Ym, Pr, L, W, F1, F2, F3, F4, Displacement\n")
        i = 0
        for line in X_emu:
            line = normal.un_nor_x(line)
            dis = Y_emu[i][0]
            f.write(
                str(line[0]) + "," + str(line[1]) + "," + str(line[2]) + "," + str(line[3]) + "," + str(line[4]) + "," + str(
                    line[5]) + "," + str(line[6]) + "," + str(line[7]) + "," + str(dis) + "\n")
            i = i + 1
    
    print('GPY database has been saved successfully')    
    
    with open('C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\Emukit_noisein\\GPY_model_with_{}.dat'.format(save_name),'wb') as f:
        joblib.dump(gpy_model, f)   
    print('GPY model has been saved successfully') 

In [ ]:
# data_number = 25
# save = '25'
# LH_generation(data_number, save)

# data_number = 40
# save = '40'
# LH_generation(data_number, save)

# data_number = 50
# save = '50'
# LH_generation(data_number, save)

# data_number = 60
# save = '60'
# LH_generation(data_number, save)

# data_number = 70
# save = '70'
# LH_generation(data_number, save)

In [ ]:
# 1%
seed=[4,14,24,34,44,54,64,74,84,94] #,44,54,64,7,17,27,34
for ele in seed:
    
    train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_15.csv'
    test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
    data_name = '15LH_15ITr'+ str(ele)
    save_name = '15LH_15ITr'+ str(ele)
    iterations = 15
    EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = ele)
    
    train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_20.csv'
    test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
    data_name = '20LH_20ITr'+str(ele)
    save_name = '20LH_20ITr'+str(ele)
    iterations = 20
    EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = ele)
    
    train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_25.csv'
    test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
    data_name = '25LH_25ITr'+str(ele)
    save_name = '25LH_25ITr'+  str(ele)
    iterations = 25
    EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = ele)

    train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_30.csv'
    test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
    data_name = '30LH_30ITr'+str(ele)
    save_name = '30LH_30ITr'+ str(ele)
    iterations = 30
    EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = ele)

    train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_40.csv'
    test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
    data_name = '40LH_40ITr'+str(ele)
    save_name = '40LH_40ITr'+ str(ele)
    iterations = 40
    EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = ele)

    train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_50.csv'
    test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
    data_name = '50LH_50ITr'+str(ele)
    save_name = '50LH_50ITr'+ str(ele)
    iterations = 50
    EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = ele)

    train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_60.csv'
    test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
    data_name = '60LH_60ITr'+str(ele)
    save_name = '60LH_60ITr'+ str(ele)
    iterations = 60
    EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = ele)

In [15]:
# 3%
seed=94 #[4,14,24,34,44,54,64,74,84,94]

train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_15.csv'
test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
data_name = '15LH_15ITr'+ str(seed)
save_name = '15LH_15ITr'+ str(seed)
iterations = 15
EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = seed)

train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_20.csv'
test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
data_name = '20LH_20ITr'+str(seed)
save_name = '20LH_20ITr'+str(seed)
iterations = 20
EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = seed)

train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_25.csv'
test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
data_name = '25LH_25ITr'+str(seed)
save_name = '25LH_25ITr'+  str(seed)
iterations = 25
EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = seed)

train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_30.csv'
test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
data_name = '30LH_30ITr'+str(seed)
save_name = '30LH_30ITr'+ str(seed)
iterations = 30
EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = seed)

train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_40.csv'
test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
data_name = '40LH_40ITr'+str(seed)
save_name = '40LH_40ITr'+ str(seed)
iterations = 40
EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = seed)

train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_50.csv'
test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
data_name = '50LH_50ITr'+str(seed)
save_name = '50LH_50ITr'+ str(seed)
iterations = 50
EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = seed)

train_data = 'C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\traindata\\final\\LH_with_xxc_60.csv'
test_data = "C:\\Users\\xuxuk\\Desktop\\xxk\\DOE\\code\\testdata_2500.csv"
data_name = '60LH_60ITr'+str(seed)
save_name = '60LH_60ITr'+ str(seed)
iterations = 60
EMU_generation(train_data, test_data, data_name, save_name, iterations, my_seed = seed)

Gaussian process regression, R2=0.53
please wait fot the iteration


 C:\Users\xuxuk\anaconda3\envs\freecad\lib\site-packages\paramz\transformations.py:111: RuntimeWarning:overflow encountered in expm1


Gaussian process regression after iterations, R2=-0.13
GPY database has been saved successfully
GPY model has been saved successfully
Gaussian process regression, R2=-5.58
please wait fot the iteration


 C:\Users\xuxuk\anaconda3\envs\freecad\lib\site-packages\paramz\transformations.py:111: RuntimeWarning:overflow encountered in expm1


Gaussian process regression after iterations, R2=-0.10
GPY database has been saved successfully
GPY model has been saved successfully
Gaussian process regression, R2=0.80
please wait fot the iteration


...

Gaussian process regression after iterations, R2=0.66
GPY database has been saved successfully
GPY model has been saved successfully
Gaussian process regression, R2=0.80
please wait fot the iteration


 C:\Users\xuxuk\anaconda3\envs\freecad\lib\site-packages\paramz\transformations.py:111: RuntimeWarning:overflow encountered in expm1


Gaussian process regression after iterations, R2=0.80
GPY database has been saved successfully
GPY model has been saved successfully
Gaussian process regression, R2=0.79
please wait fot the iteration


 C:\Users\xuxuk\anaconda3\envs\freecad\lib\site-packages\paramz\transformations.py:111: RuntimeWarning:overflow encountered in expm1


Gaussian process regression after iterations, R2=0.83
GPY database has been saved successfully
GPY model has been saved successfully
